# 衛星画像の整理

入手したPlanetの衛星画像から終了の時系列解析

In [1]:
# planetからダウンロードしたzipの解凍
import glob
import zipfile
import os

def unzip_no_pwd_folder(folder_path):
    files =  glob.glob(f"{folder_path}/*.zip")
    for file in files:
        print(file)
        with zipfile.ZipFile(file) as existing_zip:
            existing_zip.extractall(os.path.dirname(file))

In [ ]:
unzip_no_pwd_folder('/workspaces/conda_sandbox/daizu/raster')

In [18]:
import shutil
# ファイルの種類ごとにフォルダ分け
def planet_folder_move(folder_path,output_path):
    files =  glob.glob(f"{folder_path}/*_clip.xml")
    for file in files:
        print(file)    
        shutil.move(f"{file}", f"{output_path}/")
    files =  glob.glob(f"{folder_path}/*udm2_clip.tif")
    for file in files:
        print(file)    
        shutil.move(f"{file}", f"{output_path}/")
    files =  glob.glob(f"{folder_path}/*udm_clip.tif")
    for file in files:
        print(file)    
        shutil.move(f"{file}", f"{output_path}/")

In [ ]:
planet_folder_move('/workspaces/conda_sandbox/daizu/raster/files','/workspaces/conda_sandbox/daizu/raster/files/storage')

# 解析対象ファイルの作成

In [53]:
# ポリゴンだけのデータ
import geopandas as gpd
aois = gpd.read_file('/workspaces/conda_sandbox/daizu/496+120圃場.geojson')
aois = aois.drop(['Agricult_1', 'Jurisdicti', '名前_漢','LandCatego' ,'転作??','転作利?','Address_2','地権者_','Unnamed_ 1','distance','水田面?','作物等?','作物名' ], axis=1)


このファイルには収量が書き込まれていない！！　まず収量を書き足す  
このGeoJsonに3地点平均のCSVもつけてでかいファイルを作成　各カラムも調整

In [54]:
#終了とかec pHのデータ
import pandas as pd
readcsv = pd.read_csv('/workspaces/conda_sandbox/daizu/3dataAvg+収量ketugouyou .csv')

In [55]:
readcsv['圃場番号'] = readcsv['圃場番号'].astype(str)
aois['圃場番号'] = aois['uniqueID'].astype(str)

In [69]:
merged_496 = pd.merge(aois, readcsv, on='圃場番号', how='outer')

In [78]:
merged_496['unit_yield'] = merged_496['15%反収現物重量kg/10a']
merged_496.drop('15%反収現物重量kg/10a', axis=1)
merged_496[110:115]

,id,Address,CityCode,Area,作付年,H23以前,耕地面?,播種月?,sensor_loc,uniqueID,geometry,圃場番号,EC,PH,EC_mean,PH_mean,1_mean,2_mean,3_mean,4_mean,5_mean,6_mean,7_mean,8_mean,9_mean,10_mean,11_mean,12_mean,13_mean,14_mean,15_mean,16_mean,17_mean,18_mean,19_mean,20_mean,21_mean,22_mean,23_mean,24_mean,25_mean,26_mean,27_mean,28_mean,29_mean,30_mean,31_mean,32_mean,33_mean,34_mean,35_mean,36_mean,37_mean,38_mean,39_mean,40_mean,41_mean,42_mean,43_mean,44_mean,45_mean,46_mean,47_mean,48_mean,49_mean,50_mean,3データの最大圧の深さの平均,3データの最大圧力の平均,播種月日,SoilName,調査日,現物重量ｋｇ,計量時水分％,圃場面積,反収現物重量kg/10a,15%時の重量,15%反収現物重量kg/10a,unit_yield
110,05100032161-064994,秋田県八峰町峰浜塙大槻野135,53490.0,4047.0,None,?,4141.0,7月9日,NaN,63.0,"MULTIPOLYGON (((-64987.797 32215.339, -64972.0...",63.0,34.0,5.41,55.67,5.48,650.0,709.0,711.0,731.0,771.0,871.0,1206.0,1584.0,1847.0,2029.0,2074.0,2045.0,1951.0,1889.0,1840.0,1778.0,1722.0,1677.0,1651.0,1616.0,1533.0,1493.0,1550.0,1604.0,1615.0,1624.0,1637.0,1637.0,1680.0,1672.0,1710.0,1715.0,1725.0,1742.0,1744.0,1739.0,1742.0,1759.0,1818.0,1813.0,1917.0,1926.0,1929.0,1929.0,1949.0,1998.0,2034.0,2061.0,2064.0,2084.0,40.0,2492.0,7月9日,淡色普通非アロフェン質黒ボク土,12月 10日,64.0,0.194,4141.0,15.0,60.0,14.0,14.0
111,05100031778-065044,秋田県八峰町峰浜塙大槻野212,53490.0,2162.0,Ｈ25〜,None,2076.0,6月30日,2.0,68.0,"MULTIPOLYGON (((-65082.152 31784.794, -65072.4...",68.0,19.0,5.45,39.33,5.39,415.0,374.0,341.0,350.0,382.0,455.0,569.0,835.0,1221.0,1652.0,1808.0,1783.0,1713.0,1606.0,1506.0,1448.0,1431.0,1440.0,1389.0,1337.0,1414.0,1445.0,1421.0,1375.0,1390.0,1435.0,1457.0,1545.0,1637.0,1686.0,1696.0,1632.0,1603.0,1613.0,1618.0,1590.0,1547.0,1515.0,1514.0,1528.0,1533.0,1543.0,1524.0,1525.0,1578.0,1613.0,1637.0,1783.0,1812.0,1818.0,59.0,2271.0,6月30日,淡色普通非アロフェン質黒ボク土,12月 9日,201.0,0.182,2076.0,97.0,190.0,92.0,92.0
112,05100030655-065520,秋田県八峰町峰浜塙神祇林157,53490.0,1444.0,None,?,1386.0,6月29日,NaN,80.0,"MULTIPOLYGON (((-65482.344 30667.310, -65543.1...",80.0,115.0,6.07,109.33,6.07,652.0,689.0,689.0,714.0,787.0,1018.0,1329.0,1656.0,1749.0,1843.0,2052.0,2091.0,2148.0,2276.0,2069.0,1459.0,1459.0,1474.0,1508.0,1653.0,1806.0,1809.0,1686.0,1814.0,997.0,1011.0,947.0,840.0,840.0,904.0,1049.0,1049.0,1049.0,1049.0,1049.0,1049.0,1049.0,1049.0,1049.0,1049.0,1049.0,1049.0,1049.0,989.0,948.0,948.0,948.0,984.0,1011.0,1011.0,18.0,2684.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,05100032040-066069,秋田県八峰町峰浜畑谷根洗向74,53490.0,1274.0,None,?,1210.0,6月30日,NaN,20.0,"MULTIPOLYGON (((-66083.218 32063.440, -66042.7...",20.0,93.0,6.05,84.00,6.00,232.0,234.0,246.0,262.0,298.0,408.0,591.0,893.0,1130.0,1316.0,1353.0,1461.0,1462.0,1479.0,1518.0,1535.0,1579.0,1594.0,1612.0,1638.0,1643.0,1663.0,1697.0,1692.0,1719.0,1751.0,1766.0,1821.0,1830.0,1855.0,1907.0,1953.0,1968.0,1973.0,1974.0,1978.0,1980.0,1979.0,1979.0,1988.0,2143.0,2145.0,2143.0,2145.0,2146.0,2157.0,2145.0,2135.0,2132.0,2113.0,48.0,2160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,05100031895-065593,秋田県八峰町峰浜畑谷上根洗向62-1,53490.0,1294.0,None,?,1707.0,6月30日,NaN,31.0,"MULTIPOLYGON (((-65583.434 31916.840, -65581.2...",31.0,29.0,5.96,34.67,5.74,279.0,336.0,380.0,409.0,428.0,468.0,716.0,410.0,490.0,732.0,903.0,960.0,992.0,1048.0,1167.0,1265.0,1327.0,1530.0,1532.0,1502.0,1438.0,1337.0,1284.0,1238.0,1131.0,1055.0,1034.0,1035.0,1077.0,1149.0,1023.0,1705.0,1699.0,943.0,1940.0,584.0,575.0,597.0,574.0,604.0,579.0,584.0,551.0,488.0,470.0,460.0,451.0,452.0,441.0,434.0,21.0,2468.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
# 収量データだけある行だけ抽出
uniit_yield_df = merged_496.dropna(subset=['unit_yield'])
uniit_yield_df.crs

<Projected CRS: EPSG:6678>
Name: JGD2011 / Japan Plane Rectangular CS X
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Japan - zone X
- bounds: (139.49, 37.73, 142.14, 41.58)
Coordinate Operation:
- name: Japan Plane Rectangular CS zone X
- method: Transverse Mercator
Datum: Japanese Geodetic Datum 2011
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [105]:
uniit_yield_df.to_file('/workspaces/conda_sandbox/daizu/unit_yeild.geojson', driver="GeoJSON") 

In [91]:
# 23年以前という情報は115箇所分だけ　※エクセルとの突合がこの数しかできなかったから
# 作付け年の情報は7個しか残っていない
# 作付け年の情報を打つ歌目に、今もう一度この496個の圃場ポリゴンに再度エクセルとの紐付けを試みる必要がある。
merged_496.info()

115

# plot_yield_vs_eviを利用して収量とNDVIの関係を表示

次のコード　plot_yield_vs_eviに必要なデータの確認  
* aoi_path:ポリゴンパス,tif_path:衛星画像パス,,seed：？,index_name：evi ndvi,filename:outputfile
* 解凍後メタデータは削除　DN と　SR Clipの二種類画像ファイル残るがどう使い分けるのか？
* 入力ROIを一度平面直角座標系に変換して、ポリゴン縮めるバッファ与えて、地理座標に再度変換している。この平面座標系はコード動かす度に適宜変更する必要がある。
* 品種名というカラムの情報で変換対象にフィルタリングをかけている　この機能は不要なため、適宜コメントアウトする
* 収量のカラムを指定する必要がある

1シーンのEVIと収量の検討（時系列ではない）

In [106]:
!python plot_yield_vs_evi.py /workspaces/conda_sandbox/daizu/unit_yeild.geojson /workspaces/conda_sandbox/daizu/raster/files 1 ndvi /workspaces/conda_sandbox/daizu/output_vegindex

Traceback (most recent call last):
  File "rasterio/_base.pyx", line 216, in rasterio._base.DatasetBase.__init__
  File "rasterio/_shim.pyx", line 77, in rasterio._shim.open_dataset
  File "rasterio/_err.pyx", line 205, in rasterio._err.exc_wrap_pointer
rasterio._err.CPLE_OpenFailedError: '/workspaces/conda_sandbox/daizu/raster/files' not recognized as a supported file format.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/workspaces/conda_sandbox/plot_yield_vs_evi.py", line 128, in <module>
    main()
  File "/workspaces/conda_sandbox/plot_yield_vs_evi.py", line 97, in main
    evi_med = calc_evi2s(aois['geometry'], args.tif_path)
  File "/workspaces/conda_sandbox/plot_yield_vs_evi.py", line 61, in calc_evi2s
    with rio.open(tif_path) as image:
  File "/opt/conda/lib/python3.9/site-packages/rasterio/env.py", line 445, in wrapper
    return f(*args, **kwds)
  File "/opt/conda/lib/python3.9/site-packages/rasterio/__ini